In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import patches
from collections import Counter
from tqdm import tqdm
from scipy.stats import chisquare
from copy import deepcopy
from collections import deque 
import seaborn as sns


In [2]:
PARENT_DIR = '/home/groups/dpwall/briannac/SARS-CoV-2_structural_variations/'
ct_rnafold_file = PARENT_DIR + 'results/rna_structure/rna_fold/rnafold.ct'
annotated_rnafold_file = PARENT_DIR + 'results/rna_structure/rna_fold/rnafold.txt'
structure_rnafold_file_RNApdbee =  PARENT_DIR + 'results/rna_structure/RNApdbee/rnafold_pdbee.txt'
structure_rnafold_file_bpRNA = PARENT_DIR + 'results/rna_structure/bpRNA/rnafold.st'
rnafold_file_dot = PARENT_DIR + 'results/rna_structure/rna_fold/rnafold.dot'

bqseq_mxfold_file = PARENT_DIR + 'results/rna_structure/mxfold/mxfold.bpseq'
annotated_mxfold_file = PARENT_DIR + 'results/rna_structure/mxfold/mxfold_annotated.txt'
structure_mxfold_file_RNApdbee =  PARENT_DIR + 'results/rna_structure/RNApdbee/mxfold_pdbee.txt'
structure_mxfold_file_bpRNA = PARENT_DIR + 'results/rna_structure/bpRNA/mxfold.st'
mxfold_file_dot = PARENT_DIR + 'results/rna_structure/mxfold/mxfold.dot'

svs_table_file = PARENT_DIR + 'results/alignment/gisaid_seqs.svs_table_all.tsv'

# Significance of bpRNA structure

H = hairpin
B = Bulge
M = multi loop
I = Internal loop
P = Pseudoknots
S = Stems
U = Unpaired Regions

In [14]:
def Get_Structure_Counter_bpRNA(locs, bpRNA):
    sv_structure_counter = {'S': 0, 'H': 0, 'I':0, 'M':0, 'B':0, 'E':0}
    c = Counter(bpRNA[locs])
    for struct in c:
        sv_structure_counter[struct] = c[struct]
    return sv_structure_counter

In [15]:
for file_name in [structure_mxfold_file_bpRNA, structure_rnafold_file_bpRNA]:
    with open(file_name) as file:
        bprna_struct = np.array(list(file.readlines()[5].replace('\n','')))
    svs = pd.read_csv(svs_table_file)#, index_col=0)
    structure_obs = Get_Structure_Counter_bpRNA((svs['Start Pos'].values), bprna_struct)
    structure_exp = Get_Structure_Counter_bpRNA([i for i in range(100,29800)], bprna_struct)
    obs = np.array([structure_obs['S'], structure_obs['H'], structure_obs['I'], structure_obs['M'], structure_obs['B']])
    exp = np.array([structure_exp['S'], structure_exp['H'], structure_exp['I'], structure_exp['M'], structure_exp['B']])
    exp = exp/sum(exp)*sum(obs)
    idx_zero = np.where((exp!=0) | (obs!=0))[0]
    exp = exp[idx_zero]
    obs = obs[idx_zero]
    print('\n',file_name.split('/')[-2:])
    print(chisquare(obs,exp))
    print('S, H, I, M, B')
    print('obs', obs)
    print('exp', exp)


 ['bpRNA', 'mxfold.st']
Power_divergenceResult(statistic=30.998921438211987, pvalue=3.0629470341109144e-06)
S, H, I, M, B
obs [40 26 23 33  0]
exp [67.46969697 14.84538721 13.32141414 24.72861953  1.63488215]

 ['bpRNA', 'rnafold.st']
Power_divergenceResult(statistic=20.286159135219272, pvalue=0.0004384516874920986)
S, H, I, M, B
obs [59 25 19 16  3]
exp [79.99279437 13.28899963 12.26613691 12.98912421  3.46294488]


# Significance of RNApdbee annotation

In [16]:
def Get_Structure_Counter_RNApdbee(locs, structure_lists):
    sv_structure_counter = {'Loop': 0, 'Stem': 0, 'Single_strand':0}
    c = Counter([structure_lists[l] for l in locs])
    for struct in c:
        sv_structure_counter[struct] = c[struct]
    return sv_structure_counter


In [17]:
for file_name in [structure_rnafold_file_RNApdbee, structure_mxfold_file_RNApdbee]:
    with open(file_name) as f:
        structure_rnafold = f.readlines()
    structure_rnafold = [s.replace('\n', '').split(' ') for s in structure_rnafold]
    structure_list = []
    for s in structure_rnafold:
        if s=='': continue
        structure_type = s[0]
        locs = []
        for i in s:
            try:
                locs = locs + [int(i)]
            except: dummy_var=0
        structure_list = structure_list + [(structure_type, locs[::2], locs[1::2])]
    stems = [s for s in structure_list if s[0]=='Stem']
    loops = [s for s in structure_list if s[0]=='Loop']
    single_strands = [s for s in structure_list if s[0]=='Single_strand']

    stem_list = (np.concatenate([list(range(jj,kk)) for i,j,k in stems for jj,kk in zip(j,k)]))
    loop_list = (np.concatenate([list(range(jj,kk)) for i,j,k in loops for jj,kk in zip(j,k)]))
    single_strand_list = (np.concatenate([list(range(jj,kk)) for i,j,k in single_strands for jj,kk in zip(j,k)]))
    structures = ['' for i in range(1,29904)]
    for i in loop_list:
        structures[i] = 'Loop'
    for i in stem_list:
        structures[i] = 'Stem'
    for i in single_strand_list:
        structures[i] = 'Single_strand'
    
    svs = pd.read_csv(svs_table_file)#, index_col=0)
    structure_obs = Get_Structure_Counter_RNApdbee(svs['Start Pos'].values, structures)
    structure_exp = Get_Structure_Counter_RNApdbee([i for i in range(100,29800)], structures)
    obs = np.array([structure_obs['Loop'], structure_obs['Stem'], structure_obs['Single_strand']])
    exp = np.array([structure_exp['Loop'], structure_exp['Stem'], structure_exp['Single_strand']])
    exp = exp/sum(exp)*sum(obs)
    idx_zero = np.where((exp!=0) | (obs!=0))[0]
    exp = exp[idx_zero]
    obs = obs[idx_zero]
    print('\n',file_name.split('/')[-2:])
    print(chisquare(obs,exp))
    print('Loop', 'Stem', 'Single strand')
    print('obs', obs)
    print('exp', exp)


 ['RNApdbee', 'rnafold_pdbee.txt']
Power_divergenceResult(statistic=18.046719823567436, pvalue=0.00012056037275549703)
Loop Stem Single strand
obs [30 41 51]
exp [18.87097643 63.69878788 39.43023569]

 ['RNApdbee', 'mxfold_pdbee.txt']
Power_divergenceResult(statistic=15.75552682374136, pvalue=0.0003790799550758387)
Loop Stem Single strand
obs [27 33 62]
exp [18.90794613 54.58781145 48.50424242]


# Annotations for rna structure figures

In [207]:
sv_regions = [('A', 400, 800), ('B',6200, 7200), ('C',11000, 11700), ('D',20000, 22500), ('E',25000, 26500), ('F',27000, 28500)] # For #(20179, 21751)

svs = pd.read_csv(svs_table_file)

# Create color map.
insertions = svs[svs['Structural Variant']=='I']
deletions = svs[svs['Structural Variant']=='D']
deletion_ids = np.array([i[1]['Start Pos']+j  for i in deletions.iterrows() for j in range(i[1]['Length (bp)'])])
insertion_ids = np.array([i[1]['Start Pos'] for i in insertions.iterrows()])

colors = np.array(['0' for i in range(29904)])
ii = 0
for _,start, stop in sv_regions:
    ii = ii + 1
    colors[start:stop] = str(ii)
with open(out_dir + '_regions_colormap.txt', 'w') as f:
    for c in colors:
        f.write(c + '\n')
        
for region_name, start, stop in sv_regions:
    colors=color_map[start:(stop+1)]
    with open(out_dir + '_' + region_name + '_colormap.txt', 'w') as f:
        for c in colors:
            f.write(c + '\n')
            

In [219]:
for label, dot_file in [['rnafold', rnafold_file_dot]]:#, ['mxfold', mxfold_file_dot]]:
    with open(dot_file) as f:
        lines = f.readlines()
    seq=lines[0]#'...'
    struc=lines[1]#'......(((((.(((((....)))))..)))))...(((((((((((((.....))))).((((.......))))........((((((((.((.((((.(((.....))).)))))).))))))))..((((((.....)))))).(.(((((((((((..(((((...(((.((((((((.....((((((.(((((......)))))..)))))).........(((((((.((......)))))))))(((....)))))))))))))).))))).))))...))))))).).....((((((.......((..((((((...))))))..)))))))).......(((.(((((((((((((.....))).)))))..))))).)))(((((((((((((....(((((((((((..((((((((((((....((((.....)))).....((((((........)))))).....(((.(((((((((((.(((((((....(.(((.........))).).))).))))...))))..)))))).).)))......((((((((.((((.(((((..(((((((....).)))))).))))))))).)))))))).((((((((((((((((((.((((.......(((((.((.(((..(((((.(((...........)))...)))))..))).)).)))))...)))).).))))))))))..............))))).))..(((((((((((((((..(((((...((((....))))....)))).)..)))).)))).)))))))......))))))))((((((.....))))))......((((((((..(((.....(((((....))))).....))).)))))))).(((((......)))))......(((((...(((((((((...((((..(((((.((..(((((......))))).)).)))))...(((((....)))))........)))))))))))))....(((((((........)))))))(((((((...((((((((.(........)))))))))((((((((((.........((.(((((((...))))))).))..((((((....)))))).((((((((..........))))))))(((((((((.....)).)))))))((((..(((((.((.....(((((((((.(.((((...)))).).)))))))))..)).)))))..)))).....(((((...........)))))..)).)))))))).....)))))))...((((((((((((((((((((((....((((((.............((((((((((((((((((....((((((((.((((((..(((..(((((.(((((((...((((((((((((.(((.((((.((((((..((((((((((.((.((((....)))).))))))...(((((((((((.((..((.((.(((.((((...))))..))).)).))))))))))))))).((((....))))((.((..((.(((((((((......(((.....)))))))))))).))..)).)).......(((((.......((((((((((((((((((((.......)))))...((((((.(((((((((((((((((..(((((((((((....))))))))..((.....))((((((((........))))))))...........((((...((((((....))))))...))))..)))..))))))))))).)))))).))))))........((((((.(((((((((..((((....)))).)))))..))))))))))..............(((..((((((((.....(((((..((((((((((.(((((....))))))))............(((((((((.(((.(((((.((.(((...))).))))))).))).)))).)))))(((((((((((.......(((.((((((((.....)))))))).))).((((((((((...((((((((((.(..............).))))((((.(((.((((((..((((.(((((((...((((......))))))))))).)).)).)))))).))).))))((((((((((((....))))..))))))))(((((..((((..(((((....)))))....))))..)))))...........(((((((((....)))))))))((((((((...(((((((..(((((((.((...(.((((((((....)))))))))...)).)))))))..)))))))...........((((((((((((....(((..((((((((((((.((.(((.(((..(((((((..(((.(((((((((((((.....(((...(((((......)))))...)))....(((((((.....)))))))((((((((.(((....)))..(((((((((...................(((.(((((((.((((.....)))).))))))).))).))))))))).......(((((.......)))))..........(((((.......))))).(((.(((((.....(((.((((........)))).)))...(((((((((((((((((((((((((((((((.((((((.(((((........))))..(((((.((((((((.......)))))))))))))..........(((.((((((..(((.(((..((((.((((((.((((((.((((((((.(((....)))..)))))))).))))))...)))))))))).))))))..))))))))).(((((((.............(((((...((((......)))))))))((((((....))))))((((((....(((((.(((((.(((((((((..(((((((......((.((((((....((((.....))))..)))))).))......)))))))......)))))))))))))))))))..))))))..)))))))....).))))))))))).))))))))....)))))))..(((((..((((((((........)))))))))))))..((.((((.((.((.((((.(((.....(((((((.((.....)))))))))...))).))))..)).)))))).)).((((((((.((((((((...(((.....)))...)))))).)).)))))))).((((..(((..(((((((((..((((((((......))).)))))(((((((...((((...))))...)))))))..((((((.......(((...)))((((.(((((((....))))))).))))..((((.......)))).(((((((..((((((((....)))))).))..)))))))..((((((((..((((...(..((((((((((((.....(((((...))))).))))))))))))..)...))))..))).))))).......(((((((((.((((.(((......))).))))..))))))))).....))))))....(((((.((..........)).))))).((((...))))((((((..(.((((....)))).)..)))))).....(((((((..(((...)))..)))))))..))))))))).)))..)))).....)))))))))))......(((((.((...(((((((((...((((((((....))))))))...)))))))))...))..((((.(((....(((((....)))))....))).)))).((((((((........))))))))...)))))..(((((.((((......))))...)))))......(((((.((((....((((((.(((......))).)).))))....)))).))))).((((((((....(((((((((......))))..)))))....)))))))).....((((((..(((((...))))))))).))((((((((((......((((((.((.((((((......(((((.((((((((((((.......)))(((.(((((((.((.(((.(((......)))))).))...))))))).))).......)))))))))..((((......))))(((((.(((((((.((.(((((((((((((((((.((......((((((((((...)))))))))).............((((....)))).((((...(((((.((((....)))).)))))))))(((((((((.(((.(((((((((((.(((.((((..((((((((((((((((((((((((......)))))))...))))))).(((((((.(((((((((...(((((((((((((((.(((((..((((((((.((((((((........)))))))....).))))))))...)))))))))))).....(((((((..((.((.(((((((..(((....)))..)))))))...)).))..)))))))((((((((((.(((((.((((....))))..))))))))...))))))).......(((((((.(((((((((((((..((.(((........(((.(((((((((((((.((((((.....((((.(.(((((.(......).))))).).))))((((((.(((......)))))))))((((((....(((((((((.((((((((((((((((((.((((((...)))))).(((....))).......((((.....))))...))))))))))......))))))))((((((((((...))))))))))((((((.(((((((((((((....))))))..........(((((((....).)))))).(((((...)))))...((((.((((((.((((((.....)))))).)))))).)))).....................((((((..(((((((((((...))))))))))).))).)))........(((((.(((...(((((((........((((((.(.((((..(((((((((((.....((((.(((((..((.((((....)))))).....((((((((((((((........))))))....((((((((((....)))))).))))........(((((.(((.((((.......)))).)))))))).((((.(((((......(((((......)))))))))).))))......(((((.......((((((.((((.((((((((......)))))))))))).))))))..))))).(((((((........))))))).((((((((....((((((((....))))))))......))))))))...))))))))..((((((..(((.(((....))))))....)))))).......(((((((.((((((((.........)))))))))))))))(((.((((((((.((((((......)))))).........)))))..))).)))..(((((((((((((..((((((((((((((.(((....)))..))))..((((((((((((.((.((((((.(((((((((....))))))))).))))))....((((((((((.........(((((((((.....((.((.(((((......)))))))))..)))))))))((((......))))(((((((....)))))))......((((((.((((....((((((.......)))))))))))))))).((((((....((((((...(((((((............(((((((((((...........(((((.(((...(((((.((......((((((.((((((...((((((((.((....)).)))))))).)).)))).......((((((((........((((((....)))))).........(((((((...((((((.....(((.(((((((((((((((....((((.(((((((..(((.....)))..)))))))))))(((((((...)))))))....((((((((((...((........)).))).)))))))....((..((((((.(((((((.((((......)))).))))))).).))))).))........((((....((((((((....))))))))))))))))))).)))..((((((.((.(......))).))))))))))).)))....)).))))...))))))).(((...)))........((((.....))))((((((((((((.....((((.......))))...))))))))))))...)))))))).......................))))))....)))))))...))).))))).(((((.((((((.(((...((.((..(((((((((.((((.......(((((.((((((....))))..)).)))))..((((((((....((((.((((((((....)))))))))))).......))))))))...............((((((((....))))))))((((.(((((((......)))))))....))))..((((((((...........(((((...(((.((((.(((((((((...))))))))))))).)))...)))))......(((((((.(((.(((((........)))))..)))))))))).....(((((((((((((((..........))))))...........((((((((((((((((((((((...(((...((((....))))...)))((((((....))))))...(((((.(((..(((((((..(((((((((...)))))))(((((((..(((((.....)))))))))))).......................))..)).)))))..))).))))).......)))))))))))))).....))))))))..))))))))).(((........))).........(((((.((((((........)))))))))))(((((((((..((......)))))))))))...)))))))).....)))).)))))))))..))))))).)))))).))))).......))))))))))).(((((((....(((((.........(((((.((((((.....)))))).)))))....))))))))))))......)))))))..))))))..))))))..((((.(((((.........))))).))))))))))))))..((((.((.(((..((((((.(((((.......))))).))))))..)))..)).)))).)).)))).))))))))....((((((((((((((.(((((...(((((((.(((((....))))).)))))))...((((((...(((((((....)))))))...))))))..........((((((((((.(((((((((((((.(((((......))))))))))))...............))))))..)))).))))))))))).))))))))))))))((((.((((((((.(((......))))))))))).))))...((((((............((.(((...))).))(((((..(((((......)))))..)))))((((.(((((.(((((((.....)))).))).)))))))))(((((((.....)))))))......(((((((.((((((((((((((((((.....))))(((((((.(((......((.((((((((((((((((((((....)))))))).)))))))))))).)).((((((((((((((.(((.((((...((((....))))...))))))))))))))...((...))...................)))))))))).))))))).......((((...((((....))))...)))).(((((.(((...((((((((((.....))).)))))))...))).))))).....((((((((....))))))))....((((((.(((((.((..((((..(.....(((((((((((...))))))).)))))..))))..))..))))).....(((((((((((.(...((((.((((((((..((((..(((((((.......)))))))((((((................((..(((((((((((......).))))))))))..)).......((((((..........)))))))))))).........((((((.....))))))((((((.(((((((((......((((((..(((((...)))))..)))))).......)))))).....(((((.(((.(((..((((((((.....((((..(((....)))..)))).....)))))).))..))).))).)))))..))).))))))..(((..........))).((((....))))..)))))))))))))))).).)))).)))))))...))))))(((((...((((....))))..)))))......)))))))))))).)).))))))).))))))..)))))).(((((....)))))..................))))..)))))))))))))..(((.((..(((((((((.((....)).)))))))))..)).)))...))))).))))...........)))))))))))...)))).)))))))(((((....)))))...............(((((((...((((....(((.(((.(((.((((((..(((.(.(((((.............(((..(((((((((.....(((.((((....)))).))).....)))))))))..))).............)))))))))..)))))).)))))))))))).)....))))))))))))))...))))))))..))))))).))))))((((......))))..((..(((((((((((....(((((((.(..(((.(((....))).))).).))))))).((((((((((....))).))))))).......(((.((((((((.(((.((((((((((((((.(.(((((((((...((((......))))..(((((((((...(((((((.......)))))))...)))))))))((((.(((...(((((((((.(((((((.((((...)))).)))..((((((((((((...(((((((..(((.((((..((.(((((.((((....)))).)))))...))..)))).)))..)))))))....................((((((((...............))))))))............(((((((((((((((((.............)))))).........(((((......(((((((....)))))))(((..(((...((((.((((..((((.((.......)).))))...)))).)))).....)))..)))(((((((.......)))))))...(((((((((((((.((((((......)))))).))))))..)))))))))))).............(((((((...(((((((..(((((((((.((.(((....))).)))))))))))....))))))).....)))))))...)))))))))))...)))))).)))).)))))).)))))))))..))).))))......((((((..(((((((((......(((.((((.(((....))))))).))).(((((((.............((((..(((....((((...))))...)))..))))...((((.....)))).)))))))((((((((((((((..((((((...))))))))))))))...))))))))))))).))..))))))......(((((((...)))).)))))))))))).).))))))............((((((((((((((((......))))))))..)))))))))))))))).))))).)))))).))))))))))))))..))..)))))))))....))))))(((((.(((((.(((((.......))))).)))).)...)))))...(((((((((...........))))))))))))))))))))........(((.((((.((((....))))...)))).)))(((((((((.........(((((((((((.....))))))))))).((((((((.((((........((((((.......))))))...........))))..))))))))(((((.((....))))))).(((((((.......((((((((..(......).)))))))).......))...)))))..((((((..(((((.((.((((((..(((.....)))..))).))))))))))))))))(((((....((((....(((((...((....))...)))))..))))..))))).....))))))))).((((((((((..(((....((..(((.(((..(((..((((((....(((....)))..)).))))..)))..)))..)))..)))))..))))))))))..((((.((((((.(((.((((((((......)))))))).))))))))).)))).)))))))).))).....))).))..)))))))..))))..))))))))).........((((((.(((.(((((((((((((..(((....)))..)))))))).))......))).))).)))))).......((..(((((..(.....)..)))))..))...))))))))...))))))))).))...))))).))))))))))))))..))).).))))))).)))((((((...))))))..))).))))))))).)))))))))))...))))))))...)).)))))))...)))))...(((......)))........((.((((((((((.(((((((((((......(.((((.(((((.....))))).)))).).....))))))))))))))))))))).))))))).......)))))).))))))))...)))))))))).....))))).))).(((((((((((........)))))..))))))..(((((((((((((.((((.(((((((......))).)))).))))))))))).)))))).))))))))....(((((((......))))))).....((((......)))).)))))))................))))))..)))....)))))))..((((((((((((((....))))))..)))))))).....((((..(((((.....(((((((..(((((((((.....(((......))).......(((((((((((....((((((((((....((..((....))..))(((((((......)))))))...((((....))))............)))))))))).....((((((((......))))))))...((((((....(((......)))..)))))))).)))))))))..)))))).))))))))))..((((((((..((((((.((((.......)))).))))))((((..((((........(((((.......)))))........)))).))))((((.....)))).)))))))))))))..))))...)))))).))))))))))))...))..))).....)))))).)).))))...)))))))).))))))....))))))))))(((....)))...(((..(((((.(((((((............((((((..(((((....((..(((((((.........))))))))).)))))..))))))))))))).))))).)))......((((((....))))))..)))))))))))..((((((((..((((.(((((((.((((....)))).).....)))))))))).....((.(((...(((.....((((.....))))...)))...))))).((((...(((((....(((.....)))....))))).)))).....((((.((((((.((.......)).))))))))))..((.(((((...(((((((((....)))))))))..))))).))...(((((((......)))))))........(((((.((((.(((....))).)))).))))).......(((((..((((((.(((((((.((....)).))))..))).))))))...)))))((((((((((((..........))).)))))))))..........(((((((((.((((..((....))..)))))))))....)))))))))))))))))))..)))))..((((((((.((((...(((....)))..))))(((.((((..........)))).)))..(((((((.((((..((((((((....(........)...))))))))......)))).)))))))......))))))))......(((.......)))))))))))..)))..((((.((((((....)))..))).))))((((.(((((.((((...)))).)))))))))((((..(((((((((....)))))))))..)))).....((((((......)))).))((((((((((((...))))))).)))))..((((.(((((((((((((...(((.((.....)).)))...)))))))(((((.((.(((((.((.((....)).)).))))).)).)))))....)))))).))))((.....)).((((......(((((((((.(((.((((.(((((.(((((((((((((((((((.......((((....(((((...((((.....)))).))))))))).....)))))))))....(((..((((.(((((.((((..((((((....)))))).))))))......(((.((((......)))).))).....(((.(((.(((((...)))))))))))........((((((...(((.(((((.....)))))..)))...))))))(((((....)))))..((((((((.(((.((((....)))).))).....(((....)))))))))))((((((((.((.....))))))))))))).))))..))))))))))))).))).)).)))).))))))))))))......))))....((((.......))))(((((((.((((((((((((((((...(((.......)))....)))))))))..(((((.(((((.....(((((((.((((.......))))))))))).......))))).)))))........(((((((((..(((....((((....)))).))).)))))))))((((...(((((.((((.(((((.......))))).)))).)))))..))))...(((((((.(((((((((((.(((((....)))))...)))).)))).....(((((((......((((((.(((((((((((....).)))).)))))).))))))........((..(((((((..((((.......(((((.(((.....(((((((((...((....))...)))))))))...((((((((((...(((((((..((..((((((((.((.....))))))))))..)))))))))......((((((.(((((((((((((.((((.((((((..((...))..))))))...((((........)))))))).)))))).))).))))))))))....(((((.((((...(((....)))...)))).)))))..(((.((((.....)))).)))..))))))))))(((((.....))))).....((((((((.......((((.((((((.......))))))..))))))))))))))))))))..))))))))))).))....))))))).(((((.....))))).......)))))))))).((((......))))..(((((.((((((.(((.((......))...))).)))).)))))))......(((..(.((((((.(((.(((......))).))))))))))..)))(((((((.(((...(((((.......)))))....))).))))))).....))))))).)))))))....(((((.((((((((.........(((((.(((.((((((((....)))))))).....)))...)))))...)))))))))))))))))))))))))))))))))...((((.......))))(((((((.(((((((..(((((..((((((((((((.(((...((((..((.(((((((...((((.......))))...))))))).)).))))(((((..((((....))))(((((((((.......)))))))))))))).((((((((.........(((((((.....)))))))....((((((....(((((((((((((..((((((((((...(((....)))...)))))))))))))))...(((((..((((((((((((((((((...))))))))...(((...((((......((((((.((((((((((((.....((((........((((.(((.....))))))))))))))..)))))))))..))))))..))))..)))..((((((..(((.(((((...(((((......)))))))))))))...)))))).)))))...(((.((((((((((....(((((....)))))((((((((((.(((((((..(((..........))))))...)))).)))))).))))((((((((((......(((((...((((.(((((((((((.........)))))((((...))))....((((..(.((((((.((((.....))))...)))))))))))...))).))))))))))))...)))))))))).(((((.(((((((((.(((.....)))..)))).)))))..))))).)))).)))))).)))...((((((((((((((.............))).)))))..(((....)))))))))...((....))(((.(((((..((((((((.(((..........))).)))))))).))))).))).....((((((.....(((((((.((((....)))).))))))).((((....((((...))))((((((((....)))))))))))).(((((((((((.((((((((((((((((....))))))))....(((((..((......))..))).)).(((((((.......)))))))))))))...)).)))))))).)))((((((((((((((((.......))))))))......(((((...)))))(((((.(((((((((((.......((((.....((((....))))..))))...)))))))).))))))))..))))))))......((((((.((............)).))))))....))))))..((((.(((((((((.(((((((...(((...)))...))))))).)))).).)))).))))....(((..((((((........))))))..)))..........((.((((....))))))..)))))))))).....((((....))))......((((((((.(((..(((((((((((((.((((((...)))))).)))))))).)))))....(((.......)))((((((((((..(((((((....(((((((((((((((((((((((((((((...)))))))(((((.(((((............))))).))))).....))))))))..))..))))))).))........(((((((......)))))))...((((((((.((((((((......(((.((((..................)))).)))......))))))))....))))))))))).....))))))).(((((((((((....(((((((((...((((...((((((((............))))))))....)))).)))))))))......((((((((((((((((......))))))))).....(((....))))))))))..............((((.((((......).)))))))((((((......(((((((((..((.(((......))))))))))..))))((((((.......))))))(((((((...(((((((.((((.....((((.(((((((((((((..(((((...)))))(((((((((.((((.(((((.....((...(((.((((((((((((((((...)))))......)))))))).)))))).))..(((.(((((((...)))))))))).))))).).)))..)))))))))....(((((..............))))))))))).)))))).).)))))))).))))))))))))))..)))))).(((((.(((((((....).)))))))).)))......))))))))))).....))))))))))...))))))))))).(((....)))...(((((((((((((((((.((((((..(((((((...(((((.....))))).....)).)))))..))))))))))))))).....((((((((...(((((((((((((((((((((((..((......)).)))))..)))))))))))))).(((((..........))))).)))).....))))))))...(((((((((((....))))))).))))))))))))..))))))))...)))))).....((((((((.....))))))))..))))))))))).)))).)))))))))))))..))))))).))))))).....))))))...))))))...))))))).)))).))))))))..((((...)))))))).)))..))))))))..))))))....(((.......(((...)))......)))(((((((.(((...(((...)))..)))))))))).((((((((.....))))))))..((((.((((....)))).))))......))))))))..))))).)))).((((((((((((((((((((....)))))).))))))))(((((((((....(((.....)))...............(((((........(((.((((...)))))))......((.(((.......))).))(((((.......))))).....)))))((((......))))))))))))).............)))))).)))))((((.((((((...)))))).))))((((.......))))..........))))...(((((((((...)))))))))..(((......)))))))))...((((.((((((..((((.......))))..)))))).))))..((((((....))))))(((((((((((((.((((.(((.........((((((.(((.((.(.(((((((((((((((....)))))).)).))).)))))))))).))))))))).)))).........((((((.((((((((.(((((((..((((...((((.....((((((((((((.((((........)))).)))).))))....)))).......)))).........))))..)))))))...((((((((((.....))))))))))....(((....)))......)))))))).............(((((.(((.(((((((.....(((((....)))))...)))))))....)))..)))))...))))))..((((((((((.(((((.............((((((((((..............))))))))))(((((.....(((((((....))).)))).....)))))...(((((((.((..(((......))).)))))))))..........(((..(((.(((...........(((....)))))).)))..))).((((((...)))))).....(((((((...)))))))..........(((((.(((((....(((((((...((((..(((((.((.......))..)))))....))))...)))))))....))).)))))))((((((...)))))))))))))).))))))).......))))))).)))))).(((((((((((.((...((((((.(((...)))...)))))).....)).))))).)))))).)))))).....)))))))).)))).(((((((((..((.........))..))))))..)))....))))(((((((((((...))))))))))))))))..)))).)))))))))))...))))).)))))))).(((.(((...))))))......(((((((((((((........((((((((.(....((.((.((((......)))).)).))(((.(((((((((.(((..(((..(((......))).(((..(((((..((((((.(((((.(((((..(((((((.((((..(((((...)))))..))))))).))))..)).)))..)))))...(((((((......)))))))....((((((.(((((((.((((((.(((((((((....))).....(((((((((((((((....)))))..))).)))))))..)))))).))))))....))))))).(((((((((...........))))).))))))))))(..(((((((......)))))))..)......((((((...((((((((((...(((((.......)))))....))))))).......(((((((.((((((.......))))).........).)))))))(((((((...((((((((.((((.(((.((((((...)))))).))))))).)))))))).)))))))..(((((((...((((((((((((...)))).).)))))))..))))))))))..))))))...))))))..)))))..))).................((((((((..(((.(((.(.(((((....))))).).))).)))(((((...((((((..((((((....)))).))..))))))....)))))......(((((.(((((...((((((((.((......)).))))))))))))))))))((((((.......))))))......((((((((((.((.(((.((((...))))))).)).)))))))..)))..))))))))....((((((((....)))))))))))..)))))))))))).)))....((((((((((...(((...)))..(((((.((.....(..(((((.((((..(((((.....))))).))))..)))))..).....)).)))))...((((((((((......)))))).)))).((((((...(((((((..(((((...((((.(((((((.....)))))))....))))...)))))))))))).)))))))))).))))))......(((((..(((((......))))))))))).))))))))......)))))).)))))))...)))))))).((..(((.(((........((((....(((((((....)))))))((((((....)))))).....))))((((((.(((...(((((.((((((((...(((((((((((((((..((((((((((......((((((...))))))....))))))))))..)).....(((((.((((((((.(((((.....))))))))))))))))))(((..((((((..((((((...((((((...)))))).....))))))..)))).))..))).......((..((((((((((.(((((.(((............))).)))))))))))).)))..))...((((..((((((((((...)))))))))).)))))))))))))))))))))))))..))))).....)))))))))..................(((.....(((((((((((....((((((...))))))))...................((((((((((.....))))))))))...)))))))))....)))(((((.((((((((((...((((((.((((..((((((.(..(((.(........))))..).))))))..)))).)))))).............(((((..........))))).......((((((((((.((((((......)))))).....((((((.(.((((.....)))).).))))))))))))))))......(((((((((((((....)))).))))).))))...(((((.......(((...))))))))((((.(.(((.((......))))).).))))..((((((((..((((((((.(((....)))))))))))..))))))))...(((((((((....((((......(((((.((((((....)))))))))))((((((....))))))......)))).....))).))))))(((((((((((((((((((((..(((..(((((...)))))))))))))))).))).)))))))).))((((((((.(((.(((.....(((((((.((((((((.(((.....))).))))))....(((.(((((.......(((((((((((...........).))))).)))))))))).))).(((((((((..(((((((.((((...........)))).))))).((((((....(((((((........(((((((...)))))))........)))))))....))))))(((((.(((...))).)))))....(((((((..(((.((((((.(((...((((((((.((((((...)))))).))))))))((.(((((.((((((((..((((((((...))))))))...))))).(((((..(((((...))))))))))............))).))))))).......((((((((.((((.............((((((.........((((........)))).((((.((((........)))).))))...((((((.(...(((((((.(((((((((((((((((.((.......))...))))))))).((.((..((.((((....))))))))))(((((((....)))))))........(((((......))))).....(((((((((((((((.(((((((..(((...)))..))))))).))))..))).))))))))((((((((..((....)))))))))).(((.((((((.(((......((((..(((((....)))))..))))..(((((((((((((......((((.((((((((((((..((((((.....(((((((....))))))).(((((.(((((((((((((...........((((((........))))))......((((((((((((((....(((((((.((((((..................)))))))))))))(((((((......))).))))....))))).))))))))).))))((((.((((((((((....))))))).))).)))).(((((((((..(((((....((((.((......))))))..)))))..)))))))))..((((...(((((((((((((......))))).......((((((......))))))((....(((((((.............)))))))))..(((((...)))))...(((((((((.....(((...)))....))))))))).((((....))))..(((((........)))))......((((....))))))))))))...))))......((((((((((((((............((((...(((((((..(((...)))..)))))))...))))..((((((((..((...((((((((((((((((((...)))))))))).)))).)))).....((((((...(((((.........)))))...))))))(((.(((.((((..(((((((..(((((...((....))....))))).)))))))...))))))).)))..((((((.(((.((..(((((......)))))..))))))))))).(((((((((.((((((...((.(((.(((((((.....)))))))))).))...((((((((....)))))))).))))))))))))))))).))))))))...)))))))))))))).(((((...)))))))))))))))))))((((......))))(((((((((((((((..(((((((((.((((((....((((.............(((((((.((((..((((((((....))).)))))..))))......((((.((((((.(((((...(((....((((((.(((........))).))))))..)))))))))))))).))))..(((.((((..((((((..((..((((.....)))).))..))))))..)))).)))((((((((........(((((((((.............)))))))))..((((((((...((((((.((((.(((...))).))))...........((((((..(((((........)))))))))))((((....)))).(((((((...((((......))))...)))))))))))))..)))))...)))....((((((((((((.......)))................((((((((((((.....(((........))).....)))))))).)))).......)))))))))....((((((.((.(((....)))))))))))....(((((((((......)))))))))....))))))))......((((...(((((..(((((((((((.((((.........)))).))))))))))).(((((.((((((((.((((((((.(((...))))))))....(((((((................((((((((((((((((...(((((..((((((.......))))))..))))).(((((...))))))))))))))))...))))).((((((((....)))))))).)))))))((((((((..............((.((((((.(......).))))))))..))))))))......))).)))))))))))))..................((((((((.........))))))))..............(((((((((((...............(((((..(((((((.(((((((.....)))))))..))))))).))))).......(((.((((((.......))))))..)))))))))))))).))))).))))..(((((((((..((((.((..(((((.....))))).....)).))))..))))))))).)))))))..)))))).)))).)))))))))..)))).((((((....))))))(((((.((((((((...))))))))......((((((((...)))))))).....)))))...((((..((((((((........((((((...)))))).......))))).)))))))........))))))))))).))))))..))))))))))))))))((((((((((((((((..((((((((((((((((((...(((((((((((((.........))))))...........((.(((....))).))........)))))))...)))).......))))))))))...))))....)).)))))))(((((((..((((((((((.(((((((....))))))).......))))).))))).)))))))......((((((........))))))....))))))).........((((.((((....)))).))))..........)))).))))))))).((((((...........))))))..........(((((....)))))....((((((((.(.(((....)))).))))..))))..))).))))))))).....(((((((((..........((((.((((((((.....))))))))))))....))))))))).((((((((........(((((((......((((........)))).....((((((.((....))))))))..(((((((.(((((((((....)))).))))).))))))).....)))))))...(((((((.......)))).)))..((.((((((((((.(((((.((((((.(((((....))))))))))).(((((.((((((((((....)))(((..(((........)))...)))(((((.....)))))(((((((...(((((((....)))).)))...)))))))..((((((((.(((((...((((((((......((((((.........))))))((((.((((((.(((.((((((((((....))))...)))))).)))..)))))).))))..((((.((((..(((((((((((....))))))).)))).)))).))))..(((.((((((.((((((((((....))))...)))))).............((((((((((........))))))))))....)))))))))(((((((.(((..(((..((((((((.(((..(((((......)))))..(((((((...(((((..(((((((...((((((.(((((....(((.....))).)))))..))))))))))))).......(((.....))))))))...)))))))......(((.....)))..((((((.((((......((((((((.....(((((.(((((........)))))....))))).......)))))))))))).))))))...(((......)))..................((((((......((((((((((((....))))))))))))......))))))))).))))))))..)))))).))))))).....(((......)))((((((.(..........).)))))).......(((((.(((((((((.....)))))))))))))).........((((((((((...)))))))))).......((((((((.(.(..((((((((((...))))))))))..).).)))))))).((((((((((..((((((((......))))))))..)).))))))))....((((((((((((((((.....))))))))))...))))))(((.(((((((..((((((((((.(((((...))))).))))))...))))))))))).)))...((((((((((((((((((((..((((((....((((....))))..))))))..)))))))))...............((((........))))((((.(((((((((........))))))))).)))).((((((((..(((((((((........))))))))))))))))).......(((((((.....((((((...))))))......)))))))..))))))))))).))))))))(((((((((((((((.((((..(((((((......)))))))...)))).))).......(((.(((((((...)))))))..)))..(((((((.....(((((((........))))))).(((((((((..((........))))).))))))...((((((((....))))))))..((((((...))))))..........((((((((....))))))))....(((......))).........(((((...((((((.((....)).))))))..)))))........((((((.......((((((.....)))))).....))))))(((((((((...((....)).))))))))).(((((((.(((((((((((.((((((((((..((..((((...((((.(((.(((((...))))).)))..)))).....)))).))))))).))).))))))))).)))).)))))))..((((.......)))).......((((((.....))))))......)))))))......(((((..(((..(..((((((((((..(((.(((.....))))))))))).))))).)..)))..)))))(((..((((.((.((((.(((((.....((...(((((..((((((((..((((.((((((............))))))))))....)))))))))))))))))))))))).))))))..)))(((((.(((.((((..................).))).))).)))))..)))))))))))).(((((((.....)))))))(((((((((...))))).))))))))))))))).))....))))))).)))))....(((((((((.......(((.((((....)))).)))(((...))).....)))))))))))))))))))))))).)).((((.(((((((....))))))).)))).((((((.......(((((((((((....((((....(((..(((((((.(((((((...((((((..(((((.....)))))..))))))...)))))))....((((((((((....))))))))))..(((..((((((((.((.....)).))))))))))).........((((((.((((((.........((((.(((..((...((((.......))))...))..)))((((.(((((.....)))))..))))....))))..((.((((((.....)))))).))))))).).))))))(((((((.....(((.(((....((((((((...((((.(((((.((((((...))))))))))).)).))....)))))))).))).)))))))))).(((.....)))(((((.(((((.((((((((...(((((........((((((...(((((.(((.(((((.((((((......)))))).))))).)))(((..((((((....(((((..(((((.((....)).)))))..))))).......((((((((((((((((((......)))))))).)))))))..)))......(((((((........)))))))..(((((((...(((.((((((((((((........))))))))))))...)))((((((..(((...(((((..(((((.((((((.((.(((((...)))))))))))))))))))))))...(((((....((((....)))))))))................((((((((((...)))))..)))))((((...(((((((((..((((((....(((((((((.(((((...))))).)))))))))))))))...)))))))))))))...((((((((..(((.(((((((((..(((.(((((.....))))).)))((((((((((..(((((((...(((((((.((((((((.((((.(((((((((((.((((.........)))))))))..((.....))......)))))))))).))))...((((((........((((..((((..((((...(((....)))...))))..))))........(((((....(((.....)))...)))))....(((((....(((((((((.....)).)))))))..)))))............(((((((.((.(((((....))))).)).)).))))))))).(((((((....((((((((((((.........))))))))).)))(((.(((((.(((.(((........................))).)))(((....(((..........)))....)))......((((....)))).......(((....)))..(((((.....(((((((((((.(((......))).))))).)))))).....)))))......))))))))....(((((((.(((((((((((((((((...))))))))).....(((.(((((.((((((...)))))).)))))..)))))))))))..)).))))))))))))))))))........))))...((((...........))))..)))))))....).)))))).((((((((((((...))))))))))))...((((.(((((...(((((................))))).)))))))))))))))........))))....))))))))).))).......)))))))))))..))))))........(((..(((((((....)))))))..)))((((....((((((....)))))).))))...)))))))..((((.(((..((....)).)))))))....))))))..)))....((((((.......))))))..............((((...((((.....))))...))))))))).))))))..........))))).)))))))).))))).)))))((((((((......)).))))))...((((.((.((((((.((((....))))...))))))))))))(((((.((((((.......))))))...)))))......((((((.((..((((......))))..))...)))))).......((((((..((.(((((((((((..(((...((......))...))).))))))))))))))))))).))))))).)))))))((((((((.((....))..))))))))....)))))))).))).....)))))))))))))))))))))).)))))))...).)))))))))))).......(((.(....).))).(((((......)))))))))....)))).)))).))))))))).))))))))))))..))))))))).....)).))))))).))).)))))))))))))))))))))))))).))).)))..))................................'
    struc=np.array(list(struc))
    for region_name, start, stop in sv_regions:
        struc_local = deepcopy(struc[start:stop])
        idx = [(i,getIndex(''.join(struc_local),i)) for i in np.where(struc_local=='(')[0]]
        keep_idx = {i for i,j in idx if j!=-1}.union({j for i,j in idx if j!=-1})
        struc_local[[i for i in range(len(struc_local)) if i not in keep_idx]] = '.'                    
        print(region_name, label)
        print(seq[start:stop])
        print(''.join(struc_local), '\n')
#sv_regions = [(434, 829), (5978, 7308), (11218, 11329), (21962, 24781), (25031, 27312), (27383, 29661)] # For #(20179, 21751)


A rnafold
CUUAAAGAUGGCACUUGUGGCUUAGUAGAAGUUGAAAAAGGCGUUUUGCCUCAACUUGAACAGCCCUAUGUGUUCAUCAAACGUUCGGAUGCUCGAACUGCACCUCAUGGUCAUGUUAUGGUUGAGCUGGUAGCAGAACUCGAAGGCAUUCAGUACGGUCGUAGUGGUGAGACACUUGGUGUCCUUGUCCCUCAUGUGGGCGAAAUACCAGUGGCUUACCGCAAGGUUCUUCUUCGUAAGAACGGUAAUAAAGGAGCUGGUGGCCAUAGUUACGGCGCCGAUCUAAAGUCAUUUGACUUAGGCGACGAGCUUGGCACUGAUCCUUAUGAAGAUUUUCAAGAAAACUGGAACACUAAACAUAGCAGUGGUGUUACCCGUGAACUCAUGCGUGAGCUUAACG
......................................((((.....)))).....((((((........)))))).....(((.(((((((((((.(((((((....(.(((.........))).).))).))))...))))..)))))).).)))......((((((((.((((.(((((..(((((((....).)))))).))))))))).)))))))).((((((((((((((((((.((((.......(((((.((.(((..(((((.(((...........)))...)))))..))).)).)))))...)))).).))))))))))..............))))).)).....................(((...((((....))))....))) 

B rnafold
UUAAGAAAGGAGCUAAAUUGUUACAUAAACCUAUUGUUUGGCAUGUUAACAAUGCAACUAAUAAAGCCACGUAUAAACCAAAUACCUGGUGUAUACGUUGUCUUUGGAGCACAAAACCAGUUGAAACAUCAAAUUCGUUUGAUGUACUGAAGUCAGAGGACGCGCAGGGAAUGGAU

In [223]:
import seaborn as sns
sns.color_palette("husl", 6)
print(sns.color_palette("husl", 6).as_hex())


['#f77189', '#bb9832', '#50b131', '#36ada4', '#3ba3ec', '#e866f4']


In [ ]:
sv_idx=5
struc_local = copy.deepcopy(struc[sv_regions[sv_idx][1]:sv_regions[sv_idx][2]])
idx = [(i,getIndex(''.join(struc_local),i)) for i in np.where(struc_local=='(')[0]]
keep_idx = {i for i,j in idx if j!=-1}.union({j for i,j in idx if j!=-1})
struc_local[[i for i in range(len(struc_local)) if i not in keep_idx]] = '.'
print(seq[sv_regions[sv_idx][1]:sv_regions[sv_idx][2]])
print(''.join(struc_local))


In [190]:
with open(dot_file) as f:
    lines = f.readlines()
seq=lines[0]#'...'
struc=lines[1]#'......(((((.(((((....)))))..)))))...(((((((((((((.....))))).((((.......))))........((((((((.((.((((.(((.....))).)))))).))))))))..((((((.....)))))).(.(((((((((((..(((((...(((.((((((((.....((((((.(((((......)))))..)))))).........(((((((.((......)))))))))(((....)))))))))))))).))))).))))...))))))).).....((((((.......((..((((((...))))))..)))))))).......(((.(((((((((((((.....))).)))))..))))).)))(((((((((((((....(((((((((((..((((((((((((....((((.....)))).....((((((........)))))).....(((.(((((((((((.(((((((....(.(((.........))).).))).))))...))))..)))))).).)))......((((((((.((((.(((((..(((((((....).)))))).))))))))).)))))))).((((((((((((((((((.((((.......(((((.((.(((..(((((.(((...........)))...)))))..))).)).)))))...)))).).))))))))))..............))))).))..(((((((((((((((..(((((...((((....))))....)))).)..)))).)))).)))))))......))))))))((((((.....))))))......((((((((..(((.....(((((....))))).....))).)))))))).(((((......)))))......(((((...(((((((((...((((..(((((.((..(((((......))))).)).)))))...(((((....)))))........)))))))))))))....(((((((........)))))))(((((((...((((((((.(........)))))))))((((((((((.........((.(((((((...))))))).))..((((((....)))))).((((((((..........))))))))(((((((((.....)).)))))))((((..(((((.((.....(((((((((.(.((((...)))).).)))))))))..)).)))))..)))).....(((((...........)))))..)).)))))))).....)))))))...((((((((((((((((((((((....((((((.............((((((((((((((((((....((((((((.((((((..(((..(((((.(((((((...((((((((((((.(((.((((.((((((..((((((((((.((.((((....)))).))))))...(((((((((((.((..((.((.(((.((((...))))..))).)).))))))))))))))).((((....))))((.((..((.(((((((((......(((.....)))))))))))).))..)).)).......(((((.......((((((((((((((((((((.......)))))...((((((.(((((((((((((((((..(((((((((((....))))))))..((.....))((((((((........))))))))...........((((...((((((....))))))...))))..)))..))))))))))).)))))).))))))........((((((.(((((((((..((((....)))).)))))..))))))))))..............(((..((((((((.....(((((..((((((((((.(((((....))))))))............(((((((((.(((.(((((.((.(((...))).))))))).))).)))).)))))(((((((((((.......(((.((((((((.....)))))))).))).((((((((((...((((((((((.(..............).))))((((.(((.((((((..((((.(((((((...((((......))))))))))).)).)).)))))).))).))))((((((((((((....))))..))))))))(((((..((((..(((((....)))))....))))..)))))...........(((((((((....)))))))))((((((((...(((((((..(((((((.((...(.((((((((....)))))))))...)).)))))))..)))))))...........((((((((((((....(((..((((((((((((.((.(((.(((..(((((((..(((.(((((((((((((.....(((...(((((......)))))...)))....(((((((.....)))))))((((((((.(((....)))..(((((((((...................(((.(((((((.((((.....)))).))))))).))).))))))))).......(((((.......)))))..........(((((.......))))).(((.(((((.....(((.((((........)))).)))...(((((((((((((((((((((((((((((((.((((((.(((((........))))..(((((.((((((((.......)))))))))))))..........(((.((((((..(((.(((..((((.((((((.((((((.((((((((.(((....)))..)))))))).))))))...)))))))))).))))))..))))))))).(((((((.............(((((...((((......)))))))))((((((....))))))((((((....(((((.(((((.(((((((((..(((((((......((.((((((....((((.....))))..)))))).))......)))))))......)))))))))))))))))))..))))))..)))))))....).))))))))))).))))))))....)))))))..(((((..((((((((........)))))))))))))..((.((((.((.((.((((.(((.....(((((((.((.....)))))))))...))).))))..)).)))))).)).((((((((.((((((((...(((.....)))...)))))).)).)))))))).((((..(((..(((((((((..((((((((......))).)))))(((((((...((((...))))...)))))))..((((((.......(((...)))((((.(((((((....))))))).))))..((((.......)))).(((((((..((((((((....)))))).))..)))))))..((((((((..((((...(..((((((((((((.....(((((...))))).))))))))))))..)...))))..))).))))).......(((((((((.((((.(((......))).))))..))))))))).....))))))....(((((.((..........)).))))).((((...))))((((((..(.((((....)))).)..)))))).....(((((((..(((...)))..)))))))..))))))))).)))..)))).....)))))))))))......(((((.((...(((((((((...((((((((....))))))))...)))))))))...))..((((.(((....(((((....)))))....))).)))).((((((((........))))))))...)))))..(((((.((((......))))...)))))......(((((.((((....((((((.(((......))).)).))))....)))).))))).((((((((....(((((((((......))))..)))))....)))))))).....((((((..(((((...))))))))).))((((((((((......((((((.((.((((((......(((((.((((((((((((.......)))(((.(((((((.((.(((.(((......)))))).))...))))))).))).......)))))))))..((((......))))(((((.(((((((.((.(((((((((((((((((.((......((((((((((...)))))))))).............((((....)))).((((...(((((.((((....)))).)))))))))(((((((((.(((.(((((((((((.(((.((((..((((((((((((((((((((((((......)))))))...))))))).(((((((.(((((((((...(((((((((((((((.(((((..((((((((.((((((((........)))))))....).))))))))...)))))))))))).....(((((((..((.((.(((((((..(((....)))..)))))))...)).))..)))))))((((((((((.(((((.((((....))))..))))))))...))))))).......(((((((.(((((((((((((..((.(((........(((.(((((((((((((.((((((.....((((.(.(((((.(......).))))).).))))((((((.(((......)))))))))((((((....(((((((((.((((((((((((((((((.((((((...)))))).(((....))).......((((.....))))...))))))))))......))))))))((((((((((...))))))))))((((((.(((((((((((((....))))))..........(((((((....).)))))).(((((...)))))...((((.((((((.((((((.....)))))).)))))).)))).....................((((((..(((((((((((...))))))))))).))).)))........(((((.(((...(((((((........((((((.(.((((..(((((((((((.....((((.(((((..((.((((....)))))).....((((((((((((((........))))))....((((((((((....)))))).))))........(((((.(((.((((.......)))).)))))))).((((.(((((......(((((......)))))))))).))))......(((((.......((((((.((((.((((((((......)))))))))))).))))))..))))).(((((((........))))))).((((((((....((((((((....))))))))......))))))))...))))))))..((((((..(((.(((....))))))....)))))).......(((((((.((((((((.........)))))))))))))))(((.((((((((.((((((......)))))).........)))))..))).)))..(((((((((((((..((((((((((((((.(((....)))..))))..((((((((((((.((.((((((.(((((((((....))))))))).))))))....((((((((((.........(((((((((.....((.((.(((((......)))))))))..)))))))))((((......))))(((((((....)))))))......((((((.((((....((((((.......)))))))))))))))).((((((....((((((...(((((((............(((((((((((...........(((((.(((...(((((.((......((((((.((((((...((((((((.((....)).)))))))).)).)))).......((((((((........((((((....)))))).........(((((((...((((((.....(((.(((((((((((((((....((((.(((((((..(((.....)))..)))))))))))(((((((...)))))))....((((((((((...((........)).))).)))))))....((..((((((.(((((((.((((......)))).))))))).).))))).))........((((....((((((((....))))))))))))))))))).)))..((((((.((.(......))).))))))))))).)))....)).))))...))))))).(((...)))........((((.....))))((((((((((((.....((((.......))))...))))))))))))...)))))))).......................))))))....)))))))...))).))))).(((((.((((((.(((...((.((..(((((((((.((((.......(((((.((((((....))))..)).)))))..((((((((....((((.((((((((....)))))))))))).......))))))))...............((((((((....))))))))((((.(((((((......)))))))....))))..((((((((...........(((((...(((.((((.(((((((((...))))))))))))).)))...)))))......(((((((.(((.(((((........)))))..)))))))))).....(((((((((((((((..........))))))...........((((((((((((((((((((((...(((...((((....))))...)))((((((....))))))...(((((.(((..(((((((..(((((((((...)))))))(((((((..(((((.....)))))))))))).......................))..)).)))))..))).))))).......)))))))))))))).....))))))))..))))))))).(((........))).........(((((.((((((........)))))))))))(((((((((..((......)))))))))))...)))))))).....)))).)))))))))..))))))).)))))).))))).......))))))))))).(((((((....(((((.........(((((.((((((.....)))))).)))))....))))))))))))......)))))))..))))))..))))))..((((.(((((.........))))).))))))))))))))..((((.((.(((..((((((.(((((.......))))).))))))..)))..)).)))).)).)))).))))))))....((((((((((((((.(((((...(((((((.(((((....))))).)))))))...((((((...(((((((....)))))))...))))))..........((((((((((.(((((((((((((.(((((......))))))))))))...............))))))..)))).))))))))))).))))))))))))))((((.((((((((.(((......))))))))))).))))...((((((............((.(((...))).))(((((..(((((......)))))..)))))((((.(((((.(((((((.....)))).))).)))))))))(((((((.....)))))))......(((((((.((((((((((((((((((.....))))(((((((.(((......((.((((((((((((((((((((....)))))))).)))))))))))).)).((((((((((((((.(((.((((...((((....))))...))))))))))))))...((...))...................)))))))))).))))))).......((((...((((....))))...)))).(((((.(((...((((((((((.....))).)))))))...))).))))).....((((((((....))))))))....((((((.(((((.((..((((..(.....(((((((((((...))))))).)))))..))))..))..))))).....(((((((((((.(...((((.((((((((..((((..(((((((.......)))))))((((((................((..(((((((((((......).))))))))))..)).......((((((..........)))))))))))).........((((((.....))))))((((((.(((((((((......((((((..(((((...)))))..)))))).......)))))).....(((((.(((.(((..((((((((.....((((..(((....)))..)))).....)))))).))..))).))).)))))..))).))))))..(((..........))).((((....))))..)))))))))))))))).).)))).)))))))...))))))(((((...((((....))))..)))))......)))))))))))).)).))))))).))))))..)))))).(((((....)))))..................))))..)))))))))))))..(((.((..(((((((((.((....)).)))))))))..)).)))...))))).))))...........)))))))))))...)))).)))))))(((((....)))))...............(((((((...((((....(((.(((.(((.((((((..(((.(.(((((.............(((..(((((((((.....(((.((((....)))).))).....)))))))))..))).............)))))))))..)))))).)))))))))))).)....))))))))))))))...))))))))..))))))).))))))((((......))))..((..(((((((((((....(((((((.(..(((.(((....))).))).).))))))).((((((((((....))).))))))).......(((.((((((((.(((.((((((((((((((.(.(((((((((...((((......))))..(((((((((...(((((((.......)))))))...)))))))))((((.(((...(((((((((.(((((((.((((...)))).)))..((((((((((((...(((((((..(((.((((..((.(((((.((((....)))).)))))...))..)))).)))..)))))))....................((((((((...............))))))))............(((((((((((((((((.............)))))).........(((((......(((((((....)))))))(((..(((...((((.((((..((((.((.......)).))))...)))).)))).....)))..)))(((((((.......)))))))...(((((((((((((.((((((......)))))).))))))..)))))))))))).............(((((((...(((((((..(((((((((.((.(((....))).)))))))))))....))))))).....)))))))...)))))))))))...)))))).)))).)))))).)))))))))..))).))))......((((((..(((((((((......(((.((((.(((....))))))).))).(((((((.............((((..(((....((((...))))...)))..))))...((((.....)))).)))))))((((((((((((((..((((((...))))))))))))))...))))))))))))).))..))))))......(((((((...)))).)))))))))))).).))))))............((((((((((((((((......))))))))..)))))))))))))))).))))).)))))).))))))))))))))..))..)))))))))....))))))(((((.(((((.(((((.......))))).)))).)...)))))...(((((((((...........))))))))))))))))))))........(((.((((.((((....))))...)))).)))(((((((((.........(((((((((((.....))))))))))).((((((((.((((........((((((.......))))))...........))))..))))))))(((((.((....))))))).(((((((.......((((((((..(......).)))))))).......))...)))))..((((((..(((((.((.((((((..(((.....)))..))).))))))))))))))))(((((....((((....(((((...((....))...)))))..))))..))))).....))))))))).((((((((((..(((....((..(((.(((..(((..((((((....(((....)))..)).))))..)))..)))..)))..)))))..))))))))))..((((.((((((.(((.((((((((......)))))))).))))))))).)))).)))))))).))).....))).))..)))))))..))))..))))))))).........((((((.(((.(((((((((((((..(((....)))..)))))))).))......))).))).)))))).......((..(((((..(.....)..)))))..))...))))))))...))))))))).))...))))).))))))))))))))..))).).))))))).)))((((((...))))))..))).))))))))).)))))))))))...))))))))...)).)))))))...)))))...(((......)))........((.((((((((((.(((((((((((......(.((((.(((((.....))))).)))).).....))))))))))))))))))))).))))))).......)))))).))))))))...)))))))))).....))))).))).(((((((((((........)))))..))))))..(((((((((((((.((((.(((((((......))).)))).))))))))))).)))))).))))))))....(((((((......))))))).....((((......)))).)))))))................))))))..)))....)))))))..((((((((((((((....))))))..)))))))).....((((..(((((.....(((((((..(((((((((.....(((......))).......(((((((((((....((((((((((....((..((....))..))(((((((......)))))))...((((....))))............)))))))))).....((((((((......))))))))...((((((....(((......)))..)))))))).)))))))))..)))))).))))))))))..((((((((..((((((.((((.......)))).))))))((((..((((........(((((.......)))))........)))).))))((((.....)))).)))))))))))))..))))...)))))).))))))))))))...))..))).....)))))).)).))))...)))))))).))))))....))))))))))(((....)))...(((..(((((.(((((((............((((((..(((((....((..(((((((.........))))))))).)))))..))))))))))))).))))).)))......((((((....))))))..)))))))))))..((((((((..((((.(((((((.((((....)))).).....)))))))))).....((.(((...(((.....((((.....))))...)))...))))).((((...(((((....(((.....)))....))))).)))).....((((.((((((.((.......)).))))))))))..((.(((((...(((((((((....)))))))))..))))).))...(((((((......)))))))........(((((.((((.(((....))).)))).))))).......(((((..((((((.(((((((.((....)).))))..))).))))))...)))))((((((((((((..........))).)))))))))..........(((((((((.((((..((....))..)))))))))....)))))))))))))))))))..)))))..((((((((.((((...(((....)))..))))(((.((((..........)))).)))..(((((((.((((..((((((((....(........)...))))))))......)))).)))))))......))))))))......(((.......)))))))))))..)))..((((.((((((....)))..))).))))((((.(((((.((((...)))).)))))))))((((..(((((((((....)))))))))..)))).....((((((......)))).))((((((((((((...))))))).)))))..((((.(((((((((((((...(((.((.....)).)))...)))))))(((((.((.(((((.((.((....)).)).))))).)).)))))....)))))).))))((.....)).((((......(((((((((.(((.((((.(((((.(((((((((((((((((((.......((((....(((((...((((.....)))).))))))))).....)))))))))....(((..((((.(((((.((((..((((((....)))))).))))))......(((.((((......)))).))).....(((.(((.(((((...)))))))))))........((((((...(((.(((((.....)))))..)))...))))))(((((....)))))..((((((((.(((.((((....)))).))).....(((....)))))))))))((((((((.((.....))))))))))))).))))..))))))))))))).))).)).)))).))))))))))))......))))....((((.......))))(((((((.((((((((((((((((...(((.......)))....)))))))))..(((((.(((((.....(((((((.((((.......))))))))))).......))))).)))))........(((((((((..(((....((((....)))).))).)))))))))((((...(((((.((((.(((((.......))))).)))).)))))..))))...(((((((.(((((((((((.(((((....)))))...)))).)))).....(((((((......((((((.(((((((((((....).)))).)))))).))))))........((..(((((((..((((.......(((((.(((.....(((((((((...((....))...)))))))))...((((((((((...(((((((..((..((((((((.((.....))))))))))..)))))))))......((((((.(((((((((((((.((((.((((((..((...))..))))))...((((........)))))))).)))))).))).))))))))))....(((((.((((...(((....)))...)))).)))))..(((.((((.....)))).)))..))))))))))(((((.....))))).....((((((((.......((((.((((((.......))))))..))))))))))))))))))))..))))))))))).))....))))))).(((((.....))))).......)))))))))).((((......))))..(((((.((((((.(((.((......))...))).)))).)))))))......(((..(.((((((.(((.(((......))).))))))))))..)))(((((((.(((...(((((.......)))))....))).))))))).....))))))).)))))))....(((((.((((((((.........(((((.(((.((((((((....)))))))).....)))...)))))...)))))))))))))))))))))))))))))))))...((((.......))))(((((((.(((((((..(((((..((((((((((((.(((...((((..((.(((((((...((((.......))))...))))))).)).))))(((((..((((....))))(((((((((.......)))))))))))))).((((((((.........(((((((.....)))))))....((((((....(((((((((((((..((((((((((...(((....)))...)))))))))))))))...(((((..((((((((((((((((((...))))))))...(((...((((......((((((.((((((((((((.....((((........((((.(((.....))))))))))))))..)))))))))..))))))..))))..)))..((((((..(((.(((((...(((((......)))))))))))))...)))))).)))))...(((.((((((((((....(((((....)))))((((((((((.(((((((..(((..........))))))...)))).)))))).))))((((((((((......(((((...((((.(((((((((((.........)))))((((...))))....((((..(.((((((.((((.....))))...)))))))))))...))).))))))))))))...)))))))))).(((((.(((((((((.(((.....)))..)))).)))))..))))).)))).)))))).)))...((((((((((((((.............))).)))))..(((....)))))))))...((....))(((.(((((..((((((((.(((..........))).)))))))).))))).))).....((((((.....(((((((.((((....)))).))))))).((((....((((...))))((((((((....)))))))))))).(((((((((((.((((((((((((((((....))))))))....(((((..((......))..))).)).(((((((.......)))))))))))))...)).)))))))).)))((((((((((((((((.......))))))))......(((((...)))))(((((.(((((((((((.......((((.....((((....))))..))))...)))))))).))))))))..))))))))......((((((.((............)).))))))....))))))..((((.(((((((((.(((((((...(((...)))...))))))).)))).).)))).))))....(((..((((((........))))))..)))..........((.((((....))))))..)))))))))).....((((....))))......((((((((.(((..(((((((((((((.((((((...)))))).)))))))).)))))....(((.......)))((((((((((..(((((((....(((((((((((((((((((((((((((((...)))))))(((((.(((((............))))).))))).....))))))))..))..))))))).))........(((((((......)))))))...((((((((.((((((((......(((.((((..................)))).)))......))))))))....))))))))))).....))))))).(((((((((((....(((((((((...((((...((((((((............))))))))....)))).)))))))))......((((((((((((((((......))))))))).....(((....))))))))))..............((((.((((......).)))))))((((((......(((((((((..((.(((......))))))))))..))))((((((.......))))))(((((((...(((((((.((((.....((((.(((((((((((((..(((((...)))))(((((((((.((((.(((((.....((...(((.((((((((((((((((...)))))......)))))))).)))))).))..(((.(((((((...)))))))))).))))).).)))..)))))))))....(((((..............))))))))))).)))))).).)))))))).))))))))))))))..)))))).(((((.(((((((....).)))))))).)))......))))))))))).....))))))))))...))))))))))).(((....)))...(((((((((((((((((.((((((..(((((((...(((((.....))))).....)).)))))..))))))))))))))).....((((((((...(((((((((((((((((((((((..((......)).)))))..)))))))))))))).(((((..........))))).)))).....))))))))...(((((((((((....))))))).))))))))))))..))))))))...)))))).....((((((((.....))))))))..))))))))))).)))).)))))))))))))..))))))).))))))).....))))))...))))))...))))))).)))).))))))))..((((...)))))))).)))..))))))))..))))))....(((.......(((...)))......)))(((((((.(((...(((...)))..)))))))))).((((((((.....))))))))..((((.((((....)))).))))......))))))))..))))).)))).((((((((((((((((((((....)))))).))))))))(((((((((....(((.....)))...............(((((........(((.((((...)))))))......((.(((.......))).))(((((.......))))).....)))))((((......))))))))))))).............)))))).)))))((((.((((((...)))))).))))((((.......))))..........))))...(((((((((...)))))))))..(((......)))))))))...((((.((((((..((((.......))))..)))))).))))..((((((....))))))(((((((((((((.((((.(((.........((((((.(((.((.(.(((((((((((((((....)))))).)).))).)))))))))).))))))))).)))).........((((((.((((((((.(((((((..((((...((((.....((((((((((((.((((........)))).)))).))))....)))).......)))).........))))..)))))))...((((((((((.....))))))))))....(((....)))......)))))))).............(((((.(((.(((((((.....(((((....)))))...)))))))....)))..)))))...))))))..((((((((((.(((((.............((((((((((..............))))))))))(((((.....(((((((....))).)))).....)))))...(((((((.((..(((......))).)))))))))..........(((..(((.(((...........(((....)))))).)))..))).((((((...)))))).....(((((((...)))))))..........(((((.(((((....(((((((...((((..(((((.((.......))..)))))....))))...)))))))....))).)))))))((((((...)))))))))))))).))))))).......))))))).)))))).(((((((((((.((...((((((.(((...)))...)))))).....)).))))).)))))).)))))).....)))))))).)))).(((((((((..((.........))..))))))..)))....))))(((((((((((...))))))))))))))))..)))).)))))))))))...))))).)))))))).(((.(((...))))))......(((((((((((((........((((((((.(....((.((.((((......)))).)).))(((.(((((((((.(((..(((..(((......))).(((..(((((..((((((.(((((.(((((..(((((((.((((..(((((...)))))..))))))).))))..)).)))..)))))...(((((((......)))))))....((((((.(((((((.((((((.(((((((((....))).....(((((((((((((((....)))))..))).)))))))..)))))).))))))....))))))).(((((((((...........))))).))))))))))(..(((((((......)))))))..)......((((((...((((((((((...(((((.......)))))....))))))).......(((((((.((((((.......))))).........).)))))))(((((((...((((((((.((((.(((.((((((...)))))).))))))).)))))))).)))))))..(((((((...((((((((((((...)))).).)))))))..))))))))))..))))))...))))))..)))))..))).................((((((((..(((.(((.(.(((((....))))).).))).)))(((((...((((((..((((((....)))).))..))))))....)))))......(((((.(((((...((((((((.((......)).))))))))))))))))))((((((.......))))))......((((((((((.((.(((.((((...))))))).)).)))))))..)))..))))))))....((((((((....)))))))))))..)))))))))))).)))....((((((((((...(((...)))..(((((.((.....(..(((((.((((..(((((.....))))).))))..)))))..).....)).)))))...((((((((((......)))))).)))).((((((...(((((((..(((((...((((.(((((((.....)))))))....))))...)))))))))))).)))))))))).))))))......(((((..(((((......))))))))))).))))))))......)))))).)))))))...)))))))).((..(((.(((........((((....(((((((....)))))))((((((....)))))).....))))((((((.(((...(((((.((((((((...(((((((((((((((..((((((((((......((((((...))))))....))))))))))..)).....(((((.((((((((.(((((.....))))))))))))))))))(((..((((((..((((((...((((((...)))))).....))))))..)))).))..))).......((..((((((((((.(((((.(((............))).)))))))))))).)))..))...((((..((((((((((...)))))))))).)))))))))))))))))))))))))..))))).....)))))))))..................(((.....(((((((((((....((((((...))))))))...................((((((((((.....))))))))))...)))))))))....)))(((((.((((((((((...((((((.((((..((((((.(..(((.(........))))..).))))))..)))).)))))).............(((((..........))))).......((((((((((.((((((......)))))).....((((((.(.((((.....)))).).))))))))))))))))......(((((((((((((....)))).))))).))))...(((((.......(((...))))))))((((.(.(((.((......))))).).))))..((((((((..((((((((.(((....)))))))))))..))))))))...(((((((((....((((......(((((.((((((....)))))))))))((((((....))))))......)))).....))).))))))(((((((((((((((((((((..(((..(((((...)))))))))))))))).))).)))))))).))((((((((.(((.(((.....(((((((.((((((((.(((.....))).))))))....(((.(((((.......(((((((((((...........).))))).)))))))))).))).(((((((((..(((((((.((((...........)))).))))).((((((....(((((((........(((((((...)))))))........)))))))....))))))(((((.(((...))).)))))....(((((((..(((.((((((.(((...((((((((.((((((...)))))).))))))))((.(((((.((((((((..((((((((...))))))))...))))).(((((..(((((...))))))))))............))).))))))).......((((((((.((((.............((((((.........((((........)))).((((.((((........)))).))))...((((((.(...(((((((.(((((((((((((((((.((.......))...))))))))).((.((..((.((((....))))))))))(((((((....)))))))........(((((......))))).....(((((((((((((((.(((((((..(((...)))..))))))).))))..))).))))))))((((((((..((....)))))))))).(((.((((((.(((......((((..(((((....)))))..))))..(((((((((((((......((((.((((((((((((..((((((.....(((((((....))))))).(((((.(((((((((((((...........((((((........))))))......((((((((((((((....(((((((.((((((..................)))))))))))))(((((((......))).))))....))))).))))))))).))))((((.((((((((((....))))))).))).)))).(((((((((..(((((....((((.((......))))))..)))))..)))))))))..((((...(((((((((((((......))))).......((((((......))))))((....(((((((.............)))))))))..(((((...)))))...(((((((((.....(((...)))....))))))))).((((....))))..(((((........)))))......((((....))))))))))))...))))......((((((((((((((............((((...(((((((..(((...)))..)))))))...))))..((((((((..((...((((((((((((((((((...)))))))))).)))).)))).....((((((...(((((.........)))))...))))))(((.(((.((((..(((((((..(((((...((....))....))))).)))))))...))))))).)))..((((((.(((.((..(((((......)))))..))))))))))).(((((((((.((((((...((.(((.(((((((.....)))))))))).))...((((((((....)))))))).))))))))))))))))).))))))))...)))))))))))))).(((((...)))))))))))))))))))((((......))))(((((((((((((((..(((((((((.((((((....((((.............(((((((.((((..((((((((....))).)))))..))))......((((.((((((.(((((...(((....((((((.(((........))).))))))..)))))))))))))).))))..(((.((((..((((((..((..((((.....)))).))..))))))..)))).)))((((((((........(((((((((.............)))))))))..((((((((...((((((.((((.(((...))).))))...........((((((..(((((........)))))))))))((((....)))).(((((((...((((......))))...)))))))))))))..)))))...)))....((((((((((((.......)))................((((((((((((.....(((........))).....)))))))).)))).......)))))))))....((((((.((.(((....)))))))))))....(((((((((......)))))))))....))))))))......((((...(((((..(((((((((((.((((.........)))).))))))))))).(((((.((((((((.((((((((.(((...))))))))....(((((((................((((((((((((((((...(((((..((((((.......))))))..))))).(((((...))))))))))))))))...))))).((((((((....)))))))).)))))))((((((((..............((.((((((.(......).))))))))..))))))))......))).)))))))))))))..................((((((((.........))))))))..............(((((((((((...............(((((..(((((((.(((((((.....)))))))..))))))).))))).......(((.((((((.......))))))..)))))))))))))).))))).))))..(((((((((..((((.((..(((((.....))))).....)).))))..))))))))).)))))))..)))))).)))).)))))))))..)))).((((((....))))))(((((.((((((((...))))))))......((((((((...)))))))).....)))))...((((..((((((((........((((((...)))))).......))))).)))))))........))))))))))).))))))..))))))))))))))))((((((((((((((((..((((((((((((((((((...(((((((((((((.........))))))...........((.(((....))).))........)))))))...)))).......))))))))))...))))....)).)))))))(((((((..((((((((((.(((((((....))))))).......))))).))))).)))))))......((((((........))))))....))))))).........((((.((((....)))).))))..........)))).))))))))).((((((...........))))))..........(((((....)))))....((((((((.(.(((....)))).))))..))))..))).))))))))).....(((((((((..........((((.((((((((.....))))))))))))....))))))))).((((((((........(((((((......((((........)))).....((((((.((....))))))))..(((((((.(((((((((....)))).))))).))))))).....)))))))...(((((((.......)))).)))..((.((((((((((.(((((.((((((.(((((....))))))))))).(((((.((((((((((....)))(((..(((........)))...)))(((((.....)))))(((((((...(((((((....)))).)))...)))))))..((((((((.(((((...((((((((......((((((.........))))))((((.((((((.(((.((((((((((....))))...)))))).)))..)))))).))))..((((.((((..(((((((((((....))))))).)))).)))).))))..(((.((((((.((((((((((....))))...)))))).............((((((((((........))))))))))....)))))))))(((((((.(((..(((..((((((((.(((..(((((......)))))..(((((((...(((((..(((((((...((((((.(((((....(((.....))).)))))..))))))))))))).......(((.....))))))))...)))))))......(((.....)))..((((((.((((......((((((((.....(((((.(((((........)))))....))))).......)))))))))))).))))))...(((......)))..................((((((......((((((((((((....))))))))))))......))))))))).))))))))..)))))).))))))).....(((......)))((((((.(..........).)))))).......(((((.(((((((((.....)))))))))))))).........((((((((((...)))))))))).......((((((((.(.(..((((((((((...))))))))))..).).)))))))).((((((((((..((((((((......))))))))..)).))))))))....((((((((((((((((.....))))))))))...))))))(((.(((((((..((((((((((.(((((...))))).))))))...))))))))))).)))...((((((((((((((((((((..((((((....((((....))))..))))))..)))))))))...............((((........))))((((.(((((((((........))))))))).)))).((((((((..(((((((((........))))))))))))))))).......(((((((.....((((((...))))))......)))))))..))))))))))).))))))))(((((((((((((((.((((..(((((((......)))))))...)))).))).......(((.(((((((...)))))))..)))..(((((((.....(((((((........))))))).(((((((((..((........))))).))))))...((((((((....))))))))..((((((...))))))..........((((((((....))))))))....(((......))).........(((((...((((((.((....)).))))))..)))))........((((((.......((((((.....)))))).....))))))(((((((((...((....)).))))))))).(((((((.(((((((((((.((((((((((..((..((((...((((.(((.(((((...))))).)))..)))).....)))).))))))).))).))))))))).)))).)))))))..((((.......)))).......((((((.....))))))......)))))))......(((((..(((..(..((((((((((..(((.(((.....))))))))))).))))).)..)))..)))))(((..((((.((.((((.(((((.....((...(((((..((((((((..((((.((((((............))))))))))....)))))))))))))))))))))))).))))))..)))(((((.(((.((((..................).))).))).)))))..)))))))))))).(((((((.....)))))))(((((((((...))))).))))))))))))))).))....))))))).)))))....(((((((((.......(((.((((....)))).)))(((...))).....)))))))))))))))))))))))).)).((((.(((((((....))))))).)))).((((((.......(((((((((((....((((....(((..(((((((.(((((((...((((((..(((((.....)))))..))))))...)))))))....((((((((((....))))))))))..(((..((((((((.((.....)).))))))))))).........((((((.((((((.........((((.(((..((...((((.......))))...))..)))((((.(((((.....)))))..))))....))))..((.((((((.....)))))).))))))).).))))))(((((((.....(((.(((....((((((((...((((.(((((.((((((...))))))))))).)).))....)))))))).))).)))))))))).(((.....)))(((((.(((((.((((((((...(((((........((((((...(((((.(((.(((((.((((((......)))))).))))).)))(((..((((((....(((((..(((((.((....)).)))))..))))).......((((((((((((((((((......)))))))).)))))))..)))......(((((((........)))))))..(((((((...(((.((((((((((((........))))))))))))...)))((((((..(((...(((((..(((((.((((((.((.(((((...)))))))))))))))))))))))...(((((....((((....)))))))))................((((((((((...)))))..)))))((((...(((((((((..((((((....(((((((((.(((((...))))).)))))))))))))))...)))))))))))))...((((((((..(((.(((((((((..(((.(((((.....))))).)))((((((((((..(((((((...(((((((.((((((((.((((.(((((((((((.((((.........)))))))))..((.....))......)))))))))).))))...((((((........((((..((((..((((...(((....)))...))))..))))........(((((....(((.....)))...)))))....(((((....(((((((((.....)).)))))))..)))))............(((((((.((.(((((....))))).)).)).))))))))).(((((((....((((((((((((.........))))))))).)))(((.(((((.(((.(((........................))).)))(((....(((..........)))....)))......((((....)))).......(((....)))..(((((.....(((((((((((.(((......))).))))).)))))).....)))))......))))))))....(((((((.(((((((((((((((((...))))))))).....(((.(((((.((((((...)))))).)))))..)))))))))))..)).))))))))))))))))))........))))...((((...........))))..)))))))....).)))))).((((((((((((...))))))))))))...((((.(((((...(((((................))))).)))))))))))))))........))))....))))))))).))).......)))))))))))..))))))........(((..(((((((....)))))))..)))((((....((((((....)))))).))))...)))))))..((((.(((..((....)).)))))))....))))))..)))....((((((.......))))))..............((((...((((.....))))...))))))))).))))))..........))))).)))))))).))))).)))))((((((((......)).))))))...((((.((.((((((.((((....))))...))))))))))))(((((.((((((.......))))))...)))))......((((((.((..((((......))))..))...)))))).......((((((..((.(((((((((((..(((...((......))...))).))))))))))))))))))).))))))).)))))))((((((((.((....))..))))))))....)))))))).))).....)))))))))))))))))))))).)))))))...).)))))))))))).......(((.(....).))).(((((......)))))))))....)))).)))).))))))))).))))))))))))..))))))))).....)).))))))).))).)))))))))))))))))))))))))).))).)))..))................................'
struc=np.array(list(struc))

In [191]:
color_map = [str(0) for i in range(len(seq))]
for i,c in enumerate(color_map):
    if i in deletion_ids:
        color_map[i] = str(1)
    if i in insertion_ids:
        color_map[i] = str(2)
# Color map for full RNA piece.
with open(out_dir + '_colormap_all.txt', 'w') as f:
        for c in color_map:
            f.write(c + '\n')

In [197]:
def getIndex(s, i): 
    if s[i] != '(': # If input is invalid. 
        return -1
    d = deque()  # Create a deque to use it as a stack. 
    for k in range(i, len(s)): # Traverse through all elements starting from i
        # Pop a starting bracket for every closing bracket 
        if s[k] == ')': 
            d.popleft() 
        # Push all starting brackets 
        elif s[k] == '(': 
            d.append(s[i]) 
        # If deque becomes empty 
        if not d: 
            return k 
    return -1

In [172]:
sv_idx=5
struc_local = copy.deepcopy(struc[sv_regions[sv_idx][1]:sv_regions[sv_idx][2]])
idx = [(i,getIndex(''.join(struc_local),i)) for i in np.where(struc_local=='(')[0]]
keep_idx = {i for i,j in idx if j!=-1}.union({j for i,j in idx if j!=-1})
struc_local[[i for i in range(len(struc_local)) if i not in keep_idx]] = '.'
print(seq[sv_regions[sv_idx][1]:sv_regions[sv_idx][2]])
print(''.join(struc_local))


ACAUCAAGGACCUGCCUAAAGAAAUCACUGUUGCUACAUCACGAACGCUUUCUUAUUACAAAUUGGGAGCUUCGCAGCGUGUAGCAGGUGACUCAGGUUUUGCUGCAUACAGUCGCUACAGGAUUGGCAACUAUAAAUUAAACACAGACCAUUCCAGUAGCAGUGACAAUAUUGCUUUGCUUGUACAGUAAGUGACAACAGAUGUUUCAUCUCGUUGACUUUCAGGUUACUAUAGCAGAGAUAUUACUAAUUAUUAUGAGGACUUUUAAAGUUUCCAUUUGGAAUCUUGAUUACAUCAUAAACCUCAUAAUUAAAAAUUUAUCUAAGUCACUAACUGAGAAUAAAUAUUCUCAAUUAGAUGAAGAGCAACCAAUGGAGAUUGAUUAAACGAACAUGAAAAUUAUUCUUUUCUUGGCACUGAUAACACUCGCUACUUGUGAGCUUUAUCACUACCAAGAGUGUGUUAGAGGUACAACAGUACUUUUAAAAGAACCUUGCUCUUCUGGAACAUACGAGGGCAAUUCACCAUUUCAUCCUCUAGCUGAUAACAAAUUUGCACUGACUUGCUUUAGCACUCAAUUUGCUUUUGCUUGUCCUGACGGCGUAAAACACGUCUAUCAGUUACGUGCCAGAUCAGUUUCACCUAAACUGUUCAUCAGACAAGAGGAAGUUCAAGAACUUUACUCUCCAAUUUUUCUUAUUGUUGCGGCAAUAGUGUUUAUAACACUUUGCUUCACACUCAAAAGAAAGACAGAAUGAUUGAACUUUCAUUAAUUGACUUCUAUUUGUGCUUUUUAGCCUUUCUGCUAUUCCUUGUUUUAAUUAUGCUUAUUAUCUUUUGGUUCUCACUUGAACUGCAAGAUCAUAAUGAAACUUGUCACGCCUAAACGAACAUGAAAUUUCUUGUUUUCUUAGGAAUCAUCACAACUGUAGCUGCAUUUCACCAAGAAUGUAGUUUACAGUCAUGUACUCAACAUCAACCAUAUGUAGUUGAUGACCC

# Old

In [196]:
ct = pd.read_csv(ct_or_bpseq_file,  skiprows=1, header=None)
t=[np.array(x.split(' ')) for x in ct[0]]
ct = pd.DataFrame([tt[tt!=''] for tt in t])
if len(ct.columns)==6:
    ct.columns = ['loc', 'base', 'pre', 'post', 'pairs', 'loc2']
    ct['loc'] = [int(p) for p in ct['loc']]
    ct.index=ct['loc']
    ct = ct.drop(['pre', 'post', 'loc', 'loc2'], 1)
else:
    ct.columns = ['loc', 'base', 'pairs']
    ct['loc'] = [int(p) for p in ct['loc']]
    ct.index=ct['loc']
    ct = ct.drop(['loc'], 1)
ct['pairs'] = [int(p) for p in ct['pairs']]
ct = ct[ct['pairs']!=0]
ct['length'] = abs(ct['pairs']-ct.index)
ct = ct.sort_values('length')


In [559]:
#sv_regions = [(434, 829), (5978, 7308), (11218, 11329), (21962, 24781), (25031, 27312), (27383, 29661)] #(20179, 21751)

In [9]:
# Manually search for some good 'zoomed-in' sections.
for start,stop in zip(svs['Start Pos'], svs['Start Pos'] + svs['Length (bp)']):
    if stop>29903: continue
    if start<100: continue
    a=(ct[((ct.index<start) &(ct.pairs>stop) & (ct.length>100)) | (((ct.index>stop) &(ct.pairs<start)) & (ct.length>100))].iloc[0])
    print(start, stop, a.name, a.pairs, a.length)

130 131 44 20160 20116
203 204 273 171 102
222 223 273 171 102
263 264 273 171 102
508 523 829 434 395
510 519 829 434 395
514 523 829 434 395
515 521 829 434 395
515 518 829 434 395
518 521 829 434 395
519 525 829 434 395
669 672 732 631 101
686 695 732 631 101
729 738 630 747 117
1431 1434 1430 17422 15992
1577 1578 1469 17370 15901
1598 1604 1469 17370 15901
1605 1608 1469 17370 15901
2402 2429 12033 2401 9632
3333 3336 3330 3782 452
3547 3556 3410 3653 243
3852 3853 3936 3812 124
3933 3942 2676 11375 8699
4879 4882 4844 10293 5449
6311 6312 6350 6153 197
6374 6383 6143 6390 247
6501 6504 6505 6076 429
6506 6509 6021 6532 511
6510 6516 6021 6532 511
6518 6524 6021 6532 511
6684 6687 7265 6601 664
6853 6859 7252 6774 478
6876 6883 7252 6774 478
6965 6974 6956 7126 170
6996 7142 6942 7145 203
9856 9859 9600 9902 302
9858 9861 9600 9902 302
11071 11074 11079 4503 6576
11074 11077 11079 4503 6576
11075 11076 11079 4503 6576
11271 11277 4158 11330 7172
11541 11544 11554 2456 9098
12620 1

In [116]:
rna_region_length = {'A':30000, 'B':30000, 'C':30000, 'D':30000, 'E':30000, 'F':30000}
rna_region = {}
for i,j in zip(ct.index, ct.pairs):
    start  = min(i,j)
    stop = max(i,j)
    for sv in sv_regions:
        if (sv[1]>=(start))&  (sv[2]<=stop):
            if (stop-start) < rna_region_length[sv[0]]:
                rna_region_length[sv[0]] = (stop-start)
                rna_region[sv[0]] = (start, stop)

In [117]:
rna_region_length

{'A': 18505, 'B': 1330, 'C': 9562, 'D': 30000, 'E': 4815, 'F': 4815}

In [100]:
rna_region

{'B': (5978, 7308),
 'E': (24869, 29684),
 'F': (24869, 29684),
 'C': (2435, 11997),
 'A': (400, 18905)}

In [94]:
rna_region

{6200: (37, 20167),
 25000: (20169, 29871),
 27000: (20169, 29871),
 11000: (37, 20167),
 400: (37, 20167)}

In [85]:
#27383-29661
sv_idx = 1
start = 6262#sv_regions[sv_idx][1]
stop = sv_regions[sv_idx][2]
print(seq[(start-1):stop])
print(''.join(struc[(start-1):stop]))

UAAAGCCACGUAUAAACCAAAUACCUGGUGUAUACGUUGUCUUUGGAGCACAAAACCAGUUGAAACAUCAAAUUCGUUUGAUGUACUGAAGUCAGAGGACGCGCAGGGAAUGGAUAAUCUUGCCUGCGAAGAUCUAAAACCAGUCUCUGAAGAAGUAGUGGAAAAUCCUACCAUACAGAAAGACGUUCUUGAGUGUAAUGUGAAAACUACCGAAGUUGUAGGAGACAUUAUACUUAAACCAGCAAAUAAUAGUUUAAAAAUUACAGAAGAGGUUGGCCACACAGAUCUAAUGGCUGCUUAUGUAGACAAUUCUAGUCUUACUAUUAAGAAACCUAAUGAAUUAUCUAGAGUAUUAGGUUUGAAAACCCUUGCUACUCAUGGUUUAGCUGCUGUUAAUAGUGUCCCUUGGGAUACUAUAGCUAAUUAUGCUAAGCCUUUUCUUAACAAAGUUGUUAGUACAACUACUAACAUAGUUACACGGUGUUUAAACCGUGUUUGUACUAAUUAUAUGCCUUAUUUCUUUACUUUAUUGCUACAAUUGUGUACUUUUACUAGAAGUACAAAUUCUAGAAUUAAAGCAUCUAUGCCGACUACUAUAGCAAAGAAUACUGUUAAGAGUGUCGGUAAAUUUUGUCUAGAGGCUUCAUUUAAUUAUUUGAAGUCACCUAAUUUUUCUAAACUGAUAAAUAUUAUAAUUUGGUUUUUACUAUUAAGUGUUUGCCUAGGUUCUUUAAUCUACUCAACCGCUGCUUUAGGUGUUUUAAUGUCUAAUUUAGGCAUGCCUUCUUACUGUACUGGUUACAGAGAAGGCUAUUUGAACUCUACUAAUGUCACUAUUGCAACCUACUGUACUGGUUCUAUACCUUGUAGUGUUUGUCUUAGUGGUUUAGAUUCUUUAGACACCUAUCCUUCUUUAGAAACUAUACAAAUUACCAUUUC
((((((.(((((((.((((......)))).))))))).).))))).))........((((

In [86]:
for i,j,k in zip(np.cumsum(struc[(start-1):stop]==')'), np.cumsum(struc[(start-1):stop]=='('), list(range((start-1),stop))):
    print(i>j, k)

False 6261
False 6262
False 6263
False 6264
False 6265
False 6266
False 6267
False 6268
False 6269
False 6270
False 6271
False 6272
False 6273
False 6274
False 6275
False 6276
False 6277
False 6278
False 6279
False 6280
False 6281
False 6282
False 6283
False 6284
False 6285
False 6286
False 6287
False 6288
False 6289
False 6290
False 6291
False 6292
False 6293
False 6294
False 6295
False 6296
False 6297
False 6298
False 6299
False 6300
False 6301
False 6302
False 6303
False 6304
False 6305
False 6306
True 6307
True 6308
True 6309
True 6310
True 6311
True 6312
True 6313
True 6314
True 6315
True 6316
True 6317
False 6318
False 6319
False 6320
False 6321
False 6322
False 6323
False 6324
False 6325
False 6326
False 6327
False 6328
False 6329
False 6330
False 6331
False 6332
False 6333
False 6334
False 6335
False 6336
False 6337
False 6338
False 6339
False 6340
False 6341
False 6342
False 6343
False 6344
False 6345
False 6346
True 6347
True 6348
True 6349
True 6350
True 6351
True 6352
True 

In [77]:
sum(struc[(start-1):stop]=='(')

152

In [123]:
.........((((((.(((.(((((((((((((..(((....)))..)))))))).))......))).))).)))))).......((..(((((..(.....)..)))))..))....................................................................((((((...))))))..............................................................(((......)))........((.((((((((((.(((((((((((......(.((((.(((((.....))))).)))).).....))))))))))))))))))))).))


136